https://pytorch.org/tutorials/beginner/chatbot_tutorial.html

In [1]:

#import import_ipynb


%config IPCompleter.greedy=True
%matplotlib inline


import torch
import os
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import itertools
import math
import pandas as pd
import unicodedata
import codecs
import itertools



import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import numpy as np

import  pprint
pp = pprint.PrettyPrinter(indent = 4)


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

CUDA = torch.cuda.is_available()
# device = torch.device("cuda" if CUDA else "cpu")
device = torch.device("cuda")

# VOCABUOLARY


In [3]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

EOC_token = 3 # end of correct answer

class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.word2index = {'eoc': EOC_token}
        self.word2count = {'eoc': 0}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", EOC_token: "eoc"}
        self.num_words = 4  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold # CHANGE probably shouldn't do this
    def trim(self, min_count):
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)        
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", EOC_token: "eoc"}
        self.num_words = 4 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [4]:
# make data simple
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [5]:
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [6]:
def simplify_prep(s):
    rv = s.split()[1]
    if rv == 'di' or rv == 'in':
        return s.split()[2]
    else:
        return rv
    
def simplify_color(s):
    return s

def simplify_obj(s):
    s = normalizeString(s)   
    if len(s.split()) > 1:
        return s.split()[1]
    else:
        return s

#only keep first x exchanges, return pastconvo
def shorten_pastconvo(pastconvo_one, exchanges):
    numexchange = pastconvo_one.count('Tutor:')
    if numexchange > exchanges:
        return "Tutor:" + "Tutor:".join(pastconvo_one.split("Tutor:", numexchange)[-exchanges:])
#         return "Tutor:".join(pastconvo_one.split("Tutor:", exchanges+1)[:exchanges+1])
    else:
        return pastconvo_one
    
def last_exchange_pastconvo(pastconvo_one):
    exchange = shorten_pastconvo(pastconvo_one, 1)
    tutorstudent = exchange.split('Student:')
    #"Tutor:Please.."
    tutor = tutorstudent[0]
    #"ok i understand now"
    student = tutorstudent[1]
    return tutor, student
    

#added correct and translations
def construct_pastconvo(xy, exchanges, translations):
    pastconvo = []
    for i in range(len(xy)):
        pastconvo_one = shorten_pastconvo(xy['Past Convo'][i], exchanges)
        p = xy['Prep'][i]
        o = xy['Obj'][i]
        c = xy['Color'][i]
        eoc = 'EOC'
        pt = translations[simplify_prep(p)]
        ot = translations[simplify_obj(o)]
        ct = translations[simplify_color(c)]
        
        pastconvo.append(' '.join([p,o,c, pt,ot,ct, eoc, pastconvo_one]))
    return pastconvo
    

In [7]:
last_exchange_pastconvo("Tutor:one Student: ok")

('Tutor:one ', ' ok')

# DATA creating vocab


In [8]:
#dictionary of student acts
sa = pd.read_csv('./stud_acts.csv')

studresps = [str(s).strip() for s in list(sa['Student Repsonse'])]
studacts = list(sa["Actions (guess, question, affirmation, other)"])

studacts_dict = {}
for i in range(len(studacts)):
    studacts_dict[studresps[i]] = studacts[i]


In [9]:
print(studacts_dict["Sì, il gatto è dentro la scatola"])

# Sì, il gatto è dentro la scatola

[True, False, False, False]


In [19]:
xy = pd.read_csv('./prep_student_input.csv')
z = pd.read_csv('./word_translations.csv')


translations = {z['Italian'][i]: z['English'][i] for i in range(len(z))}
translations_toItalian = {z['English'][i]: z['Italian'][i] for i in range(len(z))}

EXCHANGES = 2
# concepts = list(translations.keys())
# prep = concepts[:8]
# color = concepts[8:16]
# obj = concepts[16:]


pastconvo = [normalizeString(l) for l in construct_pastconvo(xy, EXCHANGES, translations)]

response = [xy['Tutor Response'][i] for i in range(len(xy))]
response = [normalizeString(l) for l in response]

last_studresps = [last_exchange_pastconvo(p)[1] for p in list(xy['Past Convo'])]
last_studresps = [str(s).strip() for s in last_studresps]



last_studacts = []
for resp in last_studresps:
    if resp in studacts_dict:
        last_studacts.append(studacts_dict[resp])
    else:
        last_studacts.append([])
print(last_studacts[:10])
print(len(last_studacts))
print(len(pastconvo))


pairs = [[pastconvo[i], response[i]] for i in range(len(pastconvo))]

#dictionary organize 3 reponses
pairsDict = {} 
for pair in pairs:
    pairsDict[pair[0]] = []
    
for pair in pairs:
    pairsDict[pair[0]].append(pair[1])

DATA_SIZE = len(pairsDict)
INPUT_SIZE = int(DATA_SIZE * 0.9)



['[False, True, False, False]', '[True, False, False, False]', '[False, False, True, False]', '[True, False, False, False]', '[False, False, True, False]', '[False, True, False, False]', '[True, False, False, False]', '[False, False, True, False]', '[True, False, False, False]', '[False, False, True, False]']
3315
3315


In [8]:
keys = random.sample(list(pairsDict), DATA_SIZE)
values = [pairsDict[k] for k in keys]

pairs_input = []
for i in range(INPUT_SIZE):
    for j in range(len(values[i])):
        pairs_input.append([keys[i], values[i][j]])

pairs_valid = []
for i in range(INPUT_SIZE, DATA_SIZE):
    for j in range(len(values[i])):
        pairs_valid.append([keys[i], values[i][j]])

In [9]:
voc = Vocabulary("dialog")

for pair in pairs_input:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
    
for pair in pairs_valid:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
    
print("counted words:", voc.num_words)
print("vocabulary created")

counted words: 984
vocabulary created


# prepare data

In [10]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


In [11]:
#make column consistent and transpose
def zeroPadding(l, fillvalue= 0):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [12]:
def binaryMatrix(l, value=0):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [13]:
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len, input_batch, output_batch

In [14]:
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len, input_batch, output_batch = batches

# print(input_batch)
# print(output_batch)
# print("input_variable:", input_variable)
# print("lengths:", lengths)
# print("target_variable:", target_variable)
# print("mask:", mask)
# print("max_target_len:", max_target_len)


# MODEL

In [15]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

Attention using dot

In [16]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
    
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [17]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)
        
    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

# EVAL

In [18]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores
    

In [19]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    scores = [score.item() for score in scores]
    return decoded_words, scores


#returns a list of output words fiven a input sentence
def evaluateInput(encoder, decoder, searcher, voc, input_sentence, max_length):
    input_sentence = normalizeString(input_sentence)
    # Evaluate sentence
    output_words, scores = evaluate(encoder, decoder, searcher, voc, input_sentence, max_length)
    # Format and print response sentence
    # elimate everything after EOS token
    for i in range(len(output_words)):
        if output_words[i] == 'EOS':
            output_words[:] = output_words[:i]
            break
    output_words[:] = [x for x in output_words if not x == 'PAD']
#     output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    return output_words, scores
#     print('Bot:', ' '.join(output_words))

def evaluate_with_target(encoder, decoder, searcher, voc, TEST_SIZE, INPUT_SIZE):

    randomindexes = [random.randint(INPUT_SIZE, len(xy['Past Convo'])) for _ in range(TEST_SIZE)]

    inputs = [xy['Past Convo'][randomindexes[i]] for i in range(TEST_SIZE)]
    targets = [xy['Tutor Response'][randomindexes[i]] for i in range(TEST_SIZE)]

    for i in range(TEST_SIZE):
        max_length = len(normalizeString(targets[i]).split()) 
        print("past convo:", inputs[i])
        print("\n")

        output_words, scores = evaluateInput(encoder, decoder, searcher, voc, inputs[i], 20)
        print('Bot:', ' '.join(output_words))
        print("\n")
        print("Correct: ", targets[i])
        print("\n")
        
def validation_eval(encoder, decoder, searcher, voc, TEST_SIZE, INPUT_SIZE):

    randomindexes = [random.randint(INPUT_SIZE, len(xy['Past Convo'])) for _ in range(TEST_SIZE)]

    inputs = [xy['Past Convo'][randomindexes[i]] for i in range(TEST_SIZE)]
    targets = [xy['Tutor Response'][randomindexes[i]] for i in range(TEST_SIZE)]
    
    scores = []  
    for i in range(TEST_SIZE):
        max_length = len(normalizeString(targets[i]).split()) 
        print("past convo:", inputs[i])
        print("\n")

        output_words, scorelist = evaluateInput(encoder, decoder, searcher, voc, inputs[i], max_target_len)
        scores.append(sum(scorelist)/len(scorelist))
        print('Bot:', ' '.join(output_words))
        print("\n")
        
    return sum(scores)/len(scores)


# simulated student

In [20]:
import  pprint
pp = pprint.PrettyPrinter(indent = 4)


z = pd.read_csv('./word_translations.csv')
translations = {z['Italian'][i]: z['English'][i] for i in range(len(z))}
translations_toItalian = {z['English'][i]: z['Italian'][i] for i in range(len(z))}

concepts = list(translations.keys())
#TODO: dynamic
prep = concepts[:10]
color = concepts[10:18]
obj = concepts[18:]

def concept(word):
    if iseng(word):
        return translations_toItalian[word]
    else:
        return word

def iseng(word):
    if word in translations_toItalian.keys():
        return True
    else:
        return False

def isitalian(word):
    if word in translations.keys():
        return True
    else:
        return False

#guess is list in correct order, correct is solution list
def identify_incorrects(guess, correct):
    rv = []
    for i in range(len(correct)):
        if guess[i] != correct[i]:
            rv.append(correct[i])
    return rv
        
    

In [21]:

#correct order is prep obj color


class Student:
    def __init__(self, pickup, forget):
        global prep,color,obj
        self.pickup = pickup
        self.forget = forget
        self.knowledge = {
            "prep": {p: 0 for p in prep},
            "color": {c: 0 for c in color},
            "obj": {o: 0 for o in obj}
        }
       
    #TODO: can output random italian word too
    #output three words in correct order list
    def guess(self, correct):
        global translations
        #correct: [prep, object, color]
        prep_c, obj_c, color_c = correct
        prep_t = translations[prep_c]
        obj_t = translations[obj_c]
        color_t = translations[color_c]
        
#         rv = 'is it '
        rv = []
        rv.append(prep_c) if self.knowledge["prep"][prep_c] > 0.7 else rv.append(prep_t)
        rv.append(obj_c)  if self.knowledge["obj"][obj_c] > 0.7 else rv.append(obj_t)
        rv.append(color_c) if self.knowledge["color"][color_c] > 0.7 else rv.append(color_t)
        return rv
    
    #returns empty string if nothing to ask
    #returns the eng word
    def ask(self, correct):
        #TODO: randomize order
        #correct = [prep_c, obj_c, color_c]
        global translations
        word = ''
        prep_c, obj_c, color_c = correct
        
        if self.knowledge["prep"][prep_c] < 0.3:
            word = prep_c
        elif self.knowledge["obj"][obj_c] <  0.3:
            word = obj_c
        elif self.knowledge["color"][color_c] < 0.3:
            word = color_c
        
        if word != '':
            #rv = 'How do i say ' + translations[word] + ' in Italian'
            rv = translations[word]
        else:
            rv = ''
        return rv
    #forgets concepts that is not mentioned for each tutor response, excluding  gained
#     def forget(self, gained):
        
    def pickout_concepts(self, output_words):
        #return all concepts (only in italian)
        rv = []
        for word in output_words:
            if isitalian(word):
                rv.append(word)
            elif iseng(word):
                rv.append(concept(word))
        return list(set(rv))
    
    def pickout_relevent_concepts(self, output_words, correct):
        #all concepts in italian
        concepts = self.pickout_concepts(output_words)
        return [x for x in concepts if x in correct]
                
    def change_knowledge(self, output_words, correct):
        #pick out knowledge
        global prep,color,obj
        concepts = self.pickout_relevent_concepts(output_words, correct)
        p = []
        o = []
        c=  []
        for word in concepts:
            if word in prep:
                p.append(word)
            elif word in color:
                c.append(word)
            elif word in obj:
                o.append(word)
        order = ["prep", "obj", "color"]  
        groups = [p, o, c]
        change = 0
        
        #update knowledge and return change
        #TODO only forget things that are not mentioned 
        for i in range(3):
            for c in groups[i]:
                if self.knowledge[order[i]][c] < 1:
                    self.knowledge[order[i]][c] = 1
                    change +=1
#                     if random.random() < self.pickup: 
#                         self.knowledge[order[i]][c] = min(self.knowledge[order[i]][c]+0.4, 1)
#                         change += 1   

        return change




In [22]:

def penalty(output_words, student, studentexpect, correct, incorrects):
    concepts = student.pickout_concepts(output_words)
    relevent = student.pickout_relevent_concepts(output_words, correct)
    #TODO relevent includes english and italian
    
    #if nothing correct but confirmed could be because ordering or grammar
    
    #conditional on student answer
    # 1 => penalize
    # 0 => reward
    
    #incorrect = list incorrect concepts in italian
    if len(incorrects) == 0 and studentexpect == '':
        if 'correct' not in output_words or 'yes' not in output_words:
#             print("penalize: student was correct but no confirmation")
            return 1
        else:
#             print("reward: correct")
            return 0.2
    elif studentexpect != '':
        if studentexpect in relevent:
#             print("reward: answered question")
            return 0.001
        else:
#             print("penalize: didn't answer question")
            return 1
    else:
        #the student made a guess that's incorrect 
        if len(incorrects) == 3 and 'correct' in output_words:
#             print("penalize: all wrong but said correct")
            return 1
        if concepts == []:
            #maybe made a grammar hint
#             print("neutral: did not make a word hint")
            return 1
        elif len(concepts) != len(relevent):
#             print("penalize: had irrelevent hints")
            return 1
        else:
            #needs to be a hint that student got incorrect
            for hint in relevent:
                if hint not in incorrects:
#                     print("penalize: not a hint that student got wrong")
                    return 1
#             print("reward: is a hint student got wrong")
            return 0.2
        
    
    
    

In [37]:

# start one eval
def simulate_with_student(encoder, decoder, searcher, voc, max_target_len, num_convo, num_exchange):
# one conversation
    tutor_scores_perconvo = []
    for i in range(num_convo):
        sum_tutorsoftmax_oneconvo = 0
        
        student = Student(1,0)
        studentexpect = ''
        guess_incorrects = []
        prep_c =  random.choice(prep)
        color_c = random.choice(color)
        obj_c = random.choice(obj)
        correct = [prep_c, obj_c, color_c]
        correct_t = [translations[prep_c], translations[obj_c], translations[color_c]]
    #     print(correct)
        past_convo = ' '.join(correct) + ' eoc ' + ' '.join(correct_t) + ' Tutor Please try to fill in the blank in Italian.'

        #TODO: while student not correct or expects answer
        for i in range(num_exchange):
            tutorscore_oneexchange = 0
            softmax_oneexchange = 0
            tutor_log_softmax_oneexchange = 0

            coin = random.random()
            if coin < 0.8 and student.ask(correct) != '':
                askeng = student.ask(correct)
                studentexpect = translations_toItalian[askeng]
                #TODO: "what is" "what is the word for"
                input_sent = 'How do you say ' + askeng + ' in Italian'
            else:
                guess = student.guess(correct)
                guess_incorrects = identify_incorrects(guess, correct)

                if guess[0] == 'fronte':
                    guess[0] = 'di fronte'
                input_sent = "is it e "+ guess[0] + " " + guess[1] + " " + guess[2]
            past_convo += ' Student ' + input_sent
    #         print('\n')
    #         print('Student:', input_sent)
    #         print('incorrect:', guess_incorrects)
    #         print('expect:', studentexpect)

            #scores is [softmax(token) for token in model output]
            output_words, scores = evaluateInput(encoder, decoder, searcher, voc, past_convo, max_target_len)
            
            #REINFORCE
            
            softmax_oneexchange = sum(scores) / len(scores) #start off average, multiply to get probability of entire sentence, apply ln here or at the end
            #scale softmax to 0.5-1
            softmax_oneexchange = 0.5+0.5*softmax_oneexchange
            
            #REINFORCE
            output = ' '.join(output_words)
            past_convo += ' Tutor ' + output
            #evaluate on input
    #         output = input('> ')
    #         if output == 'q' or output == 'quit': break
    #         output_words = output.split()

            #how many concpets changed
            change = student.change_knowledge(output_words, correct)

            #change tutor score
            
            tutorscore_oneexchange = 1-penalty(output_words, student, studentexpect, correct, guess_incorrects)
#             tutor_log_softmax_oneexchange = tutorscore_oneexchange * np.log(softmax_oneexchange)
            tutor_log_softmax_oneexchange = 1 - tutorscore_oneexchange * softmax_oneexchange 
            
            sum_tutorsoftmax_oneconvo += tutor_log_softmax_oneexchange
            
    #         print('Bot:', output)
    #         print('pickout concpets:', student.pickout_concepts(output_words))
    #         print('change:', change)
            #TODO: tutor_score
    #         print('penalty', tutor_score)  
            studentexpect = ''
            guess_incorrects = []
        
        #REINFORCE
        tutorscore_oneconvo = sum_tutorsoftmax_oneconvo/num_exchange
        tutor_scores_perconvo.append(tutorscore_oneconvo)
        
    #one tutor score for each convo
    return sum(tutor_scores_perconvo)/len(tutor_scores_perconvo)
    
    

# test train with input

In [38]:

#simulate_with_student(encoder, decoder, searcher, voc)
# simulate_with_student(voc)

# TRAIN

In [39]:
# negative log lokelihood loss
# maskNLLLoss(decoder_output, target_variable[t], mask[t])
def maskNLLLoss(inp, target, mask): 
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    #loss = reward * loss -> 0.001/1
#     loss = loss * tutor_score
    loss = loss.to(device)
    return loss, nTotal.item()


In [40]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip, iteration, prev_tutorscore, student_simulation):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0
#     tutor_score = 3;

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
#     print("encoder output shape", encoder_outputs.shape)
#     print("last encoder hidden shape", encoder_hidden.shape)

    decoder_input = torch.LongTensor([[SOS_token for _ in range (small_batch_size)]])
    decoder_input = decoder_input.to(device)
#     print("initial decoder input shape", decoder_input.shape)
#     print(decoder_input)

    decoder_hidden =  encoder_hidden[:decoder.n_layers]
#     print("initial decoder hidden state shape", decoder_hidden.shape)
    for t in range(max_target_len):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)

        #teacher forcing
        decoder_input = target_variable[t].view(1,-1)
        #loss from
        mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
        loss += mask_loss
        print_losses.append(mask_loss.item() * nTotal)

        n_totals += nTotal
        
    returned_loss = sum(print_losses) / n_totals
    
    #student simulation
    
    if iteration > 400 and iteration % 20 == 0:
        encoder.eval()
        decoder.eval()

        searcher = GreedySearchDecoder(encoder, decoder)
        tutor_score = simulate_with_student(encoder, decoder, searcher, voc, max_target_len, 100, 2)
        encoder.train()
        decoder.train()
    else:
        tutor_score = prev_tutorscore

    if student_simulation:
        #does the scale of loss matter????
        loss = loss * tutor_score
        returned_loss = sum(print_losses) / n_totals * tutor_score
        

    
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()
    
#     print("training loss", returned_loss)
# #     print(returned_losses)
#     print("\n")

    return returned_loss, tutor_score

In [41]:
def valid(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, VALID_SIZE):
    
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
#     print("encoder output shape", encoder_outputs.shape)
#     print("last encoder hidden shape", encoder_hidden.shape)

    decoder_input = torch.LongTensor([[SOS_token for _ in range (VALID_SIZE)]])
    decoder_input = decoder_input.to(device)
#     print("initial decoder input shape", decoder_input.shape)
#     print(decoder_input)

    decoder_hidden =  encoder_hidden[:decoder.n_layers]
#     print("initial decoder hidden state shape", decoder_hidden.shape)
    for t in range(max_target_len):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)

        #teacher forcing
        decoder_input = target_variable[t].view(1,-1)

        mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
        loss += mask_loss
        print_losses.append(mask_loss.item() * nTotal)

        n_totals += nTotal
    returned_loss = sum(print_losses) / n_totals
    
#     print("validation loss", returned_loss)
# #     print(returned_losses)
#     print("\n")

    return returned_loss

In [42]:
def trainIters(voc, pairs, pairs_valid, pairs_input, hidden_size, dropout, attn_model, LEARN_RATE, WEIGHT_DECAY, encoder_n_layers, decoder_n_layers, n_iteration, small_batch_size, clip, INPUT_SIZE, student_simulation):

#     pastconvo = [p[0] for p in pairs]
#     response = [p[1] for p in pairs]
    
    embedding = nn.Embedding(voc.num_words, hidden_size)

    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = LuongAttnDecoderRNN(attn_model,  embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
    #pass models to device 
    encoder = encoder.to(device)
    decoder = decoder.to(device)

    encoder.train()
    decoder.train()

    #optimizers
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=LEARN_RATE, weight_decay=WEIGHT_DECAY)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=LEARN_RATE, weight_decay=WEIGHT_DECAY)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # start training
    print("START TRAINING")
    print("\n")

    returned_losses = []
    validation_losses = []
    tutor_scores = []
    searcher = GreedySearchDecoder(encoder, decoder)
    best_valid_loss = 1000000
    
    valid_loss = 7

    #predefine eval set
    # randomindexes = [random.randint(INPUT_SIZE, len(pastconvo)) for _ in range(TEST_SIZE)]
    # evalset = [pairs[randomindexes[i]] for i in range(len(randomindexes))]
    VALID_SIZE = len(pairs_valid)

    iteration = 0
    tutor_score = 1
    # for iteration in range(num_iters):
    
    while iteration <= n_iteration:
        iteration += 1
#         print(" Iteration:", iteration)

        randomindexes = [random.randint(0, INPUT_SIZE) for _ in range(small_batch_size)]
        trainset = [pairs_input[randomindexes[i]] for i in range(small_batch_size)]

        batches = batch2TrainData(voc, trainset)
        input_variable, lengths, target_variable, mask, max_target_len, input_batch, output_batch = batches

        #train one batch

        returned_loss, tutor_score = train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer, small_batch_size, clip,iteration, tutor_score, student_simulation)

        returned_losses.append(returned_loss)
        tutor_scores.append(tutor_score)
    #     print(returned_losses)

#         print("\n")

        if iteration % 20 == 0:
            encoder.eval()
            decoder.eval()

            evalset = pairs_valid

            evalbatches = batch2TrainData(voc, evalset)
            input_variable, lengths, target_variable, mask, max_target_len, input_batch, output_batch = evalbatches

            valid_loss = valid(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                         decoder, embedding, encoder_optimizer, decoder_optimizer, VALID_SIZE, clip, VALID_SIZE)
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                
            encoder.train()
            decoder.train()
        
        #stop training if validation loss stops decreasing
        if len(validation_losses) > 1000 and valid_loss > validation_losses[-60] + 0.1:
            break;
        
        validation_losses.append(valid_loss)
        

        if iteration % 400 == 0:   
            plt.plot(returned_losses, color='blue')
            plt.plot(validation_losses, color='red')
            plt.plot([t*10 for t in tutor_scores], color='green')
            plt.show()
    
    return encoder, decoder, best_valid_loss
    



In [ ]:
n_iteration = 2700
clip = 50.0

# hidden_options = [128]
# dropout_options = [0.1, 0.5]
# attn_models = ['dot']
# learnrate_options = [0.001]
# weightdecay_options = [0.0005]
# layer_options = [2]
# batch_options = [4]




# params = [(h, d, a, l, w, el, dl, b) for h in hidden_options for d in dropout_options 
#           for a in attn_models for l in learnrate_options for w in weightdecay_options for el in layer_options for dl in layer_options for b in batch_options]

# best_model = [params[0], 100000]


# for p in params:
#     print("hidden dropout attn leaning weight layers batchsize")
#     print(p)
    
#     hidden_size = p[0]
#     dropout = p[1]
#     attn_model = p[2]
#     LEARN_RATE = p[3]
#     WEIGHT_DECAY = p[4]
#     encoder_n_layers = p[5]
#     decoder_n_layers = p[6]
#     small_batch_size = p[7]
    
#     encoder, decoder, valid_loss = trainIters(voc, pairs, pairs_valid, pairs_input, hidden_size, dropout, attn_model, 
#                             LEARN_RATE, WEIGHT_DECAY, encoder_n_layers, decoder_n_layers, n_iteration, small_batch_size, clip, INPUT_SIZE, False)
    
#     print(valid_loss)
#     print('\n')
    
    
#     if valid_loss < best_model[1]:
#         best_model = [p, valid_loss]

# print("best model", best_model)


#run on one model
hidden_size = 128
dropout = 0.1
attn_model = 'dot'
LEARN_RATE = 0.001
WEIGHT_DECAY = 0.0005
encoder_n_layers = 2
decoder_n_layers = 2
small_batch_size = 4

# encoder, decoder, valid_loss = trainIters(voc, pairs, pairs_valid, pairs_input, hidden_size, dropout, attn_model, 
#                                           LEARN_RATE, WEIGHT_DECAY, encoder_n_layers, decoder_n_layers, n_iteration, small_batch_size, clip, INPUT_SIZE, False)




# print(valid_loss)

e, d, val = trainIters(voc, pairs, pairs_valid, pairs_input, hidden_size, dropout, attn_model, 
                                          LEARN_RATE, WEIGHT_DECAY, encoder_n_layers, decoder_n_layers, n_iteration, small_batch_size, clip, INPUT_SIZE, True)
print(val)



In [ ]:
e2, d2, val2 = trainIters(voc, pairs, pairs_valid, pairs_input, hidden_size, dropout, attn_model, 
                                          LEARN_RATE, WEIGHT_DECAY, encoder_n_layers, decoder_n_layers, n_iteration, small_batch_size, clip, INPUT_SIZE, False)
print(val2)

EVALUATION on validation set

In [ ]:
# TEST_SIZE = 30

# e2.eval()
# d2.eval()

# searcher = GreedySearchDecoder(e2, d2)

# evaluate_with_target(e2, d2, searcher, voc, TEST_SIZE, INPUT_SIZE)

In [ ]:
#evaluate student simulation
e2.eval()
d2.eval()

e.eval()
d.eval()

searcher2 = GreedySearchDecoder(e2, d2)

no_student_score = simulate_with_student(e2, d2, searcher2, voc, max_target_len, 1000)

searcher = GreedySearchDecoder(e, d)

with_student_score = simulate_with_student(e, d, searcher, voc, max_target_len, 1000)

print(no_student_score)
print(with_student_score)
